In [1]:
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [2]:
function fib_n(n::Integer)
    d = Dict(zero(n)=>big"0", one(n)=>big"1")
    fib_n(n, d)
end
function fib_n(n, d)
    if haskey(d, n)
        return d[n]
    end
    if n < 0
        result = iseven(n) ? -fib_n(-n, d) : fib_n(-n, d)
        d[n] = result
        return result
    end
    m = n ÷ 2
    result = if iseven(n)
        (2 * fib_n(m - 1, d) + fib_n(m, d)) * fib_n(m, d)
    else
        fib_n(m, d) ^ 2 + fib_n(m + 1, d) ^ 2
    end
    d[n] = result
    return result
end

fib_n (generic function with 2 methods)

In [3]:
struct FibType end
const Fib = FibType()

FibType()

In [4]:
Base.getindex(::FibType, index::Integer) = fib_n(index)

In [5]:
struct FibRange{R<:AbstractRange{<:Integer}}
    range::R
end

In [6]:
Base.getindex(::FibType, r::AbstractRange{<:Integer}) = FibRange(r)

In [7]:
Base.IteratorEltype(::Type{<:FibRange}) = Base.HasEltype()
Base.eltype(::Type{<:FibRange}) = BigInt

In [8]:
Base.IteratorSize(::Type{<:FibRange}) = Base.HasShape{1}()
Base.length(fr::FibRange) = length(fr.range)
Base.size(fr::FibRange) = size(fr.range)

In [9]:
function Base.iterate(fr::FibRange{<:UnitRange})
    index = fr.range.start
    start = Fib[index]
    prev = Fib[index - 1]
    len = length(fr)
    iterate(fr, (start, prev, len))
end
function Base.iterate(fr::FibRange{<:UnitRange}, (current, prev, len)::Tuple{BigInt, BigInt, Int})
    len <= 0 && return nothing
    (current, (current + prev, current, len - 1))
end

In [10]:
collect(Fib[1:10])

10-element Array{BigInt,1}:
  1
  1
  2
  3
  5
  8
 13
 21
 34
 55

In [11]:
Fib[1:10] .+ Fib[2:11]

10-element Array{BigInt,1}:
   2
   3
   5
   8
  13
  21
  34
  55
  89
 144

In [12]:
Fib[1:10] .+ (1:10)

10-element Array{BigInt,1}:
  2
  3
  5
  7
 10
 14
 20
 29
 43
 65

In [13]:
2 .* Fib[1:10]

10-element Array{BigInt,1}:
   2
   2
   4
   6
  10
  16
  26
  42
  68
 110

In [14]:
using BenchmarkTools

In [15]:
@benchmark Fib[1:10] .+ Fib[2:11]

BenchmarkTools.Trial: 
  memory estimate:  4.38 KiB
  allocs estimate:  86
  --------------
  minimum time:     2.109 μs (0.00% GC)
  median time:      2.453 μs (0.00% GC)
  mean time:        6.034 μs (40.07% GC)
  maximum time:     8.372 ms (87.84% GC)
  --------------
  samples:          10000
  evals/sample:     9